In [44]:
import pandas as pd

dados_datatran = []
for ii in range(2017,2020):
    csv = './acidentes-brasil-prf/dados/datatran' + str(ii) + '.csv' 
  
  dado = pd.read_csv(csv, encoding = "ISO-8859-1", sep=';', engine='python', error_bad_lines=False)
  dados_datatran.append(dado)
  
datatran = pd.concat(dados_datatran, ignore_index=True)

In [131]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline

br_acidentes_count = datatran.br.dropna().astype(int).astype(str).value_counts().sort_values(ascending=False)
br_acidentes_count.index = 'BR ' + br_acidentes_count.index 

br_acidentes_bar = go.Bar(x=br_acidentes_count.index,
                          y=br_acidentes_count.values,
                          marker=dict(color='rgb(180, 20, 20)'))

data_quantidade_bar = [br_acidentes_bar]

layout_quantidade_bar = go.Layout(title="Acidentes em Rodovias Federais",
        font={'size':10},
        xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Rodovias Federais',font=dict(size=18))),
        yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Quantidade de Acidentes',font=dict(size=18,)))
    )

fig_quantidade_bar = go.Figure(data=data_quantidade_bar,layout=layout_quantidade_bar)
offline.plot(fig_quantidade_bar,filename='bar_br_count.html')

'bar_br_count.html'

In [160]:
import folium
import requests
import geocoder
#datatran[datatran.br == datatran.br.dropna().value_counts().head(5).index[1]].uf.value_counts()

headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

response = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/"+
                        "?formato=application/vnd.geo+json&resolucao=2",
                       headers=headers)
brazil_json = response.json()


brazil_geo = geocoder.arcgis("Brazil")
m = folium.Map(location=brazil_geo.latlng,zoom_start=4,tiles='Stamen Terrain')

folium.Choropleth(geo_data=brazil_json, line_weight=2, fill_opacity=0, name='Marcação das UFs').add_to(m)
m